In [ ]:
# default_exp gan.learner

# gan.learner

> API details.

TODO maik: add simple synthetic data?!

In [ ]:
# hide
# export
from tqdm.notebook import tqdm, tnrange
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from fastai.torch_core import set_seed
import pandas as pd

In [ ]:
# export
# TODO maik: should be a class with a constructur and a fit function
# TODO maik: remove col_name_ws
# TODO maik: can the device be removed?
def train(epochs, dls, Dis, Gen, dis_optimizer, gen_optimizer, col_name_ws, device, n_noise_samples, cols):
    set_seed(123123)
    n_samples = 1000
    rand_values = torch.rand(n_samples, n_noise_samples)

    dis_real_losses, dis_fake_losses, gen_losses = [],[],[]
    # Gen = Gen.to(device)
    # TODO maik: check if data is shuffled. e.g.:
#      if torch.allclose(dl.one_batch()[1], dl.one_batch()[1]):
#         raise ValueError(
#             "Dataset does not to be seemed shuffle, which is required for training."
#         )
    
    for epoch in tnrange(epochs, desc="epochs"):

        for i, (noise_sample,real_input_batch) in enumerate(dls[0]):
            noise_sample = noise_sample.to(device)
            real_input_batch = real_input_batch.to(device)
            minibatchsize = len(real_input_batch)

    #         ## train the discriminator (i-times per epoch):
            # TODO maik: should be configurable
            for i in range(3):
                Dis.zero_grad()
                ## train on real input
                dis_real = Dis(real_input_batch)

                ## train on generated input
                dis_fake = Dis(Gen(noise_sample).detach())
                dis_loss = -(torch.mean(dis_real) - torch.mean(dis_fake))
                dis_loss.backward()
                dis_optimizer.step()


                ## clamp the discriminator parameter
                for p in Dis.parameters():
                    # TODO: what amount of clamp to use?
                    # TODO maik: should be configurable
                    p.data.clamp_(-0.1, 0.1)

            dis_real_losses.append(dis_real.detach().mean())
            dis_fake_losses.append(dis_fake.detach().mean())
            # TODO maik: should be configurable
            for i in range(1):            
                Gen.zero_grad()
                gen_out = Gen(noise_sample)
                dis_gen = Dis(gen_out)
                gen_loss = -torch.mean(dis_gen)
                gen_loss.backward()
                gen_optimizer.step()

            gen_losses.append(gen_loss.detach().mean())

        disl = dis_loss.detach().mean()
        genl = gen_loss.detach().mean()
        # TODO maik: improve output of trainer
        print(f"{epoch} dis_loss {disl} gen_loss {genl} ")
        
        # TODO maik: should be configurable
#         if epoch % 5 == 0:
            # TODO maik: an optional plotting function provided by the user should be configurable. 
            # e.g. it should be redirected to a plotting function by the user
#             with torch.no_grad():
#                 fake_samples = Gen(rand_values.to(device))

#             df_fake = pd.DataFrame(fake_samples, columns = cols)
#             df_real = to.train.items

#             sns.histplot(df_real[col_name_ws], label="WR", color="grey", bins=50, stat="probability")# 
#             sns.histplot(df_fake[col_name_ws], label="WF", color="green", bins=50, stat="probability")# stat="probability")
#             sns.histplot(df_real["PowerGeneration"], label="PR", color="blue", bins=50, stat="probability")# stat="probability")
#             sns.histplot(df_fake["PowerGeneration"], label="PF", color="yellow", bins=50, stat="probability")# stat="probability")
#             plt.title(f"Epoch {epoch}")
#             plt.legend()
#             plt.show()

#             plt.scatter(to.train.items[col_name_ws], to.train.items.PowerGeneration, alpha=0.25, label="real")
#             plt.scatter(df_fake[col_name_ws], df_fake.PowerGeneration, alpha=0.25, label="fake")
#             plt.legend()
#             plt.title(f"Epoch {epoch}")
#             plt.show()
    
    # TODO maik: rather store as attributes in the class?!
    return dis_real_losses, dis_fake_losses